Collect and clean data

In [3]:
# import pandas and numpy
import pandas as pd
import numpy as np

In [13]:
# Concatenate each individual year csv to a single csv
# created a single df named soi
soi = pd.DataFrame()
# create variable specifiy columns for soi df
id_cols = ['COUNTYFIPS', 'COUNTYNAME', 'STATEFIPS']
# use for loop to concatenate each individual year csv & calculate SALT amount per capita
for year in range(10,18):
# get the file name
    file_name = f'{year}incyallnoagi.csv'
# read the file individual year csv file
    tmp = pd.read_csv(file_name, encoding='latin-1')
# list columns of year csv file
    list(tmp.columns)
# calculate the SALT amount per capita
    tmp['salt_amount_per_cap'] = (
            tmp['A18425'] + tmp['A18450'] + tmp['A18500']) / tmp['N1']
# specify the columns to keep and add a SALT per capita column
    tmp = tmp[id_cols + ['salt_amount_per_cap']]
# add a column specifying the year
    tmp['Year'] = int(f"20{year}")
# select only the rows that are provide data on the county level 
    tmp = tmp.loc[tmp['COUNTYFIPS'] != 0]
# concatenate the data calculated for the year to the single df
    soi = pd.concat([soi, tmp], axis=0, ignore_index=True)
# sort values of the new columns by county and year
    soi = soi.sort_values(by=id_cols + ['Year'],
                      ascending=True)
# write new combined df to a csv
soi.to_csv('all_years_soi.csv', index=False)

In [ ]:
# the all year csv into df
soi = pd.read_csv('all_years_soi.csv')
# group by county and calculate the mean SALT amount per capita across across the years
soi_avg = soi.groupby(['COUNTYFIPS', 'STATEFIPS']).mean()[['salt_amount_per_cap']]
# reset the index based on the grouping
soi_avg = soi_avg.reset_index()
# cast df column data type to string to keep the datatype consistent
soi_avg['STATEFIPS']=soi_avg['STATEFIPS'].astype(str)
# add zeros to beginning of string to keep length of string consistently 2
soi_avg['STATEFIPS']=soi_avg['STATEFIPS'].str.zfill(2)
# create a duplicate STATEFIPS column
soi_avg['STATEFIPS2']=soi_avg['STATEFIPS']
# cast column data type to string to keep the datatype consistent
soi_avg['COUNTYFIPS']=soi_avg['COUNTYFIPS'].astype(str)
# add zeros to beginning of string to keep length of string consistently 3
soi_avg['COUNTYFIPS']=soi_avg['COUNTYFIPS'].str.zfill(3)
# create area_fips column by combining statefips with countyfips
soi_avg['area_fips']=soi_avg[['STATEFIPS2', 'COUNTYFIPS']].apply(lambda x: ''.join(x), axis=1)
# rename db soi_avg_pre to incluce area_fips, statefips, salt_amount_per_cap
soi_avg_pre = soi_avg[['area_fips','STATEFIPS', 'salt_amount_per_cap']]




In [ ]:
# create df1 read in master.csv (wage)
df1=pd.read_csv('master.csv')

In [4]:
# create df2 set equal to soi_avg_pre (SALT)
df2=soi_avg_pre
# set df3 equal to df1 and df2 merged on area_fips
df3=pd.merge(df1, df2, on="area_fips")

In [14]:
# delete unwanted columns from merge
df3=df3.drop("Unnamed: 0", axis=1)

In [16]:
# write the modified df3 to csv as wage_salt.csv
df3.to_csv('wage_salt.csv', index=False)

In [2]:
# read wage_salt.csv to verify write, as anchor point for next steps
df3=pd.read_csv('wage_salt.csv')

Having cleaned the data.  Now we will group and organise the data.

In [3]:
# rename STATEFIPS to state_fips for naming convention consistency
df3.rename(columns = {"STATEFIPS": "state_fips"},
            inplace = True)

In [4]:
# create a boolean variable 'treated' true if SALT per capita > 10
df3['treated'] = np.where(df3.salt_amount_per_cap > 10, 1, 0)


In [5]:
# create boolean variable 'post' for after 2017 policy change
df3['post'] = np.where(df3.year > 2017, 1, 0)

In [5]:

# create column treated*post to signify both SALT > 10 and post 2017 policy
df3['treated*post'] = df3.post * df3.treated

In [7]:
# keep only rows that are not statewide agregates
df3=df3.loc[df3['industry_code'] != 10]

In [9]:
# write modified df3 to new version of csv as new anchor point
df3.to_csv('wage_salt_v2.csv', index=False)

Data has now been cleaned and organised.  We will now run regressions. 

In [ ]:
# instatll python packages
pip install statsmodels
pip install linearmodels

In [24]:
# import libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import PooledOLS


In [5]:
# if restarting the project read in the csv as df3
df3=pd.read_csv('wage_salt_v2.csv')

In [ ]:
# if restarting the project check the data
df3.head()

In [6]:
# change year datatype to categorical
year = pd.Categorical(df3.year)
# set index based on area_fips and year
df3 = df3.set_index(['area_fips', 'year'])
df3['year'] = year


In [7]:
# change state datatype to categorical
state = pd.Categorical(df3.state_fips)
df3['state'] = state

In [8]:
# change industry datatype to categorical
industry = pd.Categorical(df3.industry_code)
df3['industry'] = industry

In [16]:
# create data frame with only rows have total annual wages of 0 (ie federal govt)
df_wage_zero=df3.loc[df3['total_annual_wages'] == 0]
len(df_wage_zero.index) 

17463132

In [17]:
# keep only the rows where total annual wages is not zero
df3=df3.loc[df3['total_annual_wages'] != 0]

In [15]:
# check to see which industries have total annual wage of 0
df_wage_zero.head()

industry_code             area_title           own_title  \
area_fips year                                                            
17095     2021         92611  Knox County, Illinois  Federal Government   
          2021        926110  Knox County, Illinois  Federal Government   

                                                   industry_title  \
area_fips year                                                      
17095     2021  NAICS 92611 Administration of general economic...   
          2021  NAICS 926110 Administration of general economi...   

                annual_avg_emplvl  total_annual_wages  state_fips  \
area_fips year                                                      
17095     2021                  0                  10          17   
          2021                  0                  10          17   

                salt_amount_per_cap  treated  post  treated*post  year state  \
area_fips year                                                                 
17095     2021             1.325571        0     1             0  2021    17   
          2021             1.325571        0     1             0  2021    17   

               industry  
area_fips year           
17095     2021    92611  
          2021   926110

In [14]:
# check arbitrarily picked section of df where annual wage is zero
df5=df3.iloc[100000:2000000]

In [18]:
# check tail section of df where annual wage is zero
df5.tail()

,industry_code,area_title,own_title,industry_title,annual_avg_emplvl,total_annual_wages,state_fips,salt_amount_per_cap,treated,post,treated*post,state,industry,year
1999995,923120,"San Diego County, California",State Government,NAICS 923120 Administration of public health p...,0,0,6,4.963774,0,1,0,6,923120,2018
1999996,92313,"San Diego County, California",Federal Government,NAICS 92313 Other human resource programs admi...,372,33977974,6,4.963774,0,1,0,6,92313,2018
1999997,92313,"San Diego County, California",State Government,NAICS 92313 Other human resource programs admi...,983,76855234,6,4.963774,0,1,0,6,92313,2018
1999998,92313,"San Diego County, California",Local Government,NAICS 92313 Other human resource programs admi...,0,0,6,4.963774,0,1,0,6,92313,2018
1999999,923130,"San Diego County, California",Federal Government,NAICS 923130 Other human resource programs adm...,372,33977974,6,4.963774,0,1,0,6,923130,2018


In [26]:
# check datatypes of columns
df3.dtypes

industry_code            object
area_title               object
own_title                object
industry_title           object
annual_avg_emplvl         int64
total_annual_wages        int64
state_fips                int64
salt_amount_per_cap     float64
treated                   int64
post                      int64
treated*post              int64
year                   category
state                  category
industry               category
dtype: object

In [28]:
# take the log of wage
logwage = np.log(df3.total_annual_wages)

In [27]:
# ordinary least squares regression of logwage as a function of treatment
FE_ols = smf.ols(formula='logwage ~ 1 + treated*post + C(year)',
data = df3).fit()
print(FE_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                 1.184e+04
Date:                Sat, 19 Nov 2022   Prob (F-statistic):               0.00
Time:                        16:52:25   Log-Likelihood:            -2.1608e+07
No. Observations:            10158863   AIC:                         4.322e+07
Df Residuals:                10158851   BIC:                         4.322e+07
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          15.0007      0.002   72

In [ ]:
# run the same regression adding state as a independent variable
FE_ols = smf.ols(formula='logwage ~ 1 + treated*post + C(year) + C(state)',
data = df3).fit()
print(FE_ols.summary())